# Permutation-based variable importance (PVI) - Homework 5 - Karol Pustelnik

In this notebook, we will focus on the permutation-based variable importance (PVI) method. We will use the PVI method to analyze the importance of the features in the heart disease dataset. We will compare results across many models and discuss the differences and similarities. I will mainly focus on one model - xgboost - with different settings of hyperparameters and variable transformations. We will also discuss the differences between the PVI and the SHAP tree method.


# Theoretical background
Permutation-based variable importance (PVI)


Permutation feature importance is a model inspection technique that can be used for any fitted estimator when the data is tabular. This is especially useful for non-linear or opaque estimators. The permutation feature importance is defined to be the decrease in a model score when a single feature value is randomly shuffled. This procedure breaks the relationship between the feature and the target, thus the drop in the model score is indicative of how much the model depends on the feature. This technique benefits from being `model agnostic` and can be calculated many times with different permutations of the feature.

source: https://scikit-learn.org/stable/modules/permutation_importance.html




In [268]:
import dalex as dx
import xgboost
import shap
import sklearn
import pandas as pd
import numpy as np
import warnings
import alibi
import plotly.express as px
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")
import plotly.io as pio
pio.renderers.default = "notebook"
import matplotlib.pyplot as plt


In [ ]:
# Import data
data = pd.read_csv('heart.csv')
data_org = pd.read_csv('heart.csv')

# One hot encoding of categorical features

#data = pd.get_dummies(data, columns = ['sex', 'cp', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall'])

# Splitting the data

y = data['output'] # Target variable
X = data.drop(['output'], axis=1) # Features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xgboost
model = xgboost.XGBClassifier(
    n_estimators=200, 
    max_depth=4, 
)

model.fit(X_train, y_train)

# Logistic regression
log_reg = LogisticRegression() 
log_reg.fit(X_train, y_train) # Fitting model




# Model evaluation

In [ ]:
def pf_xgboost_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] =\
        df.select_dtypes(['object'])\
        .apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]

explainer = dx.Explainer(model, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)

In [271]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.78125,0.862069,0.819672,0.819672,0.912716


# Permutation-based Variable Importance for xgboost

I will now calculate the permutation-based variable importance for the xgboost model. The settings are as follows:

* xgboost model
* data without transformations
* n_estimators = 200
* max_depth = 4
* the rest of the parameters are the default




In [272]:
pvi = explainer.model_parts(random_state=0)

In [273]:
pvi.result

,variable,dropout_loss,label
0,chol,0.076509,XGBClassifier
1,exng,0.079095,XGBClassifier
2,oldpeak,0.079957,XGBClassifier
3,fbs,0.085884,XGBClassifier
4,_full_model_,0.087284,XGBClassifier
5,trtbps,0.088470,XGBClassifier
6,thalachh,0.088470,XGBClassifier
7,restecg,0.092672,XGBClassifier
8,thall,0.094935,XGBClassifier
9,slp,0.100108,XGBClassifier


## Results interpretation

The interpretation of the results of the method is straightforward. The values correspond to the decrease in the model score when the feature is randomly shuffled. The higher the value of the feature importance, the more important the feature is for the model. 

* The most important feature of the model is `caa` - the number of major vessels (0-4). After shuffling the values of this feature, the model loss is increased by 0.032. This is the highest value of the feature importance.
* Another important feature is `cp` - chest pain type. After shuffling the values of this feature, the model loss is increased by 0.03. This is the second highest value of the feature importance.
* The third most important feature is `sex`. After shuffling the values of this feature, the model loss is increased by 0.021.

In this particular result, we can see that permutation of some features led to an `increase in model performance`. This is true for features:

* `chol` - cholestoral in mg/dl
* `exng` - exercise induced angina (1 = yes; 0 = no)
* `oldpeak` - ST depression induced by exercise relative to rest

It suggests, that the model is not very sensitive to these features. It is also possible that the model is overfitting to these features.

Overall most of the features are important for the model.

In [ ]:
pvi.plot(show=False).update_layout(autosize=False, width=600, height=450)

<img src="vip1.png">

What we need to have in mind while interpreting the results, is that the PVI method is stochastic. It means that for different seeds, we may see different results. To quantify the variability of the results, we can calculate some basic statistics of the feature importance.

# Experiment - calculate results for different seeds

I will now calculate the results for different seeds. I will calculate the mean and standard deviation of the feature importance and other interesting statistics. I will use the same settings as in the previous experiment.

In [278]:
def experiment(explainer, n_samples = 100):
    pvi = explainer.model_parts(random_state=0)
    df = pvi.result.transpose()
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[1:2], columns=headers)
    for seed in range(1, n_samples):
        pvi = explainer.model_parts(random_state=seed)
        df = pvi.result.transpose()
        headers = df.iloc[0]
        df = pd.DataFrame(df.values[1:2], columns=headers)
        new_df = new_df.append(df)
    
    new_df = new_df.astype(float)
    new_df = new_df.drop(['_baseline_'], axis=1)
    new_df.reset_index(drop=True, inplace=True)
    
    means_basic = new_df.mean().sort_values(ascending=True)
    
    return means_basic, new_df

In [279]:
means_no_scaling, new_df_no_scaling = experiment(explainer, n_samples = 100)

In [280]:
fig_boxplot = px.box(new_df_no_scaling, y=means_no_scaling.index , title='Boxplot of PVI for XGBoost model')
fig_means = px.bar(means_no_scaling, title='Mean PVI for XGBoost model')

In [ ]:
fig_boxplot.show()
fig_means.update_layout(showlegend=False)

<img src="fig2.png">

<img src="fig3.png">

# Experiment results interpretation

After running the method 100 times, we can see that the results are quite stable. The mean and standard deviation of the feature importance is quite low, except for features: `thall` and `caa` where the standard deviation is quite high. 

On average, the most important feature is `cp` - chest pain type. 
The second most important feature is `caa` - the number of major vessels (0-4).

Only two features fall below the full model performance: `chol` and `fbs`. This suggests that the model is not very sensitive to these features. It is also possible that the model is overfitting these features.

## Comparison with previous results

The results are quite similar to the previous results. The top 6 features are the same. This time, however, the order of the features is different. 

When it comes to the features that fall below the full model performance, the feature `chol` is still the least important feature. The feature `fbs` is now the second least important feature, but in the previous experiment, it was just above the full model performance. 

If we were to select features based on the PVI method, we would select all the features except for `chol` and `fbs`.



# Comparison across models

Now I will compare the results of the PVI method across many models. I will consider the following models:

* xgboost:
  * previous settings
  * data scaled

* logistic regression:
  * default settings

* random forest:
  * default settings

In [ ]:
# Xgboost with data scaling
scaler = MinMaxScaler()
data[['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']] = scaler.fit_transform(data[['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']])
y = data['output'] # Target variable
X = data.drop(['output'], axis=1) # Features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xgboost
model_scaled = xgboost.XGBClassifier(
    n_estimators=200, 
    max_depth=4, 
)

model_scaled.fit(X_train, y_train)

explainer_scaled = dx.Explainer(model_scaled, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)

In [ ]:
# Logistic regression - default settings

log_reg = LogisticRegression() 
log_reg.fit(X_train, y_train) 

explainer_log = dx.Explainer(log_reg, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)


In [ ]:
# Random forest - default settings
forest_model = sklearn.ensemble.RandomForestClassifier()
forest_model.fit(X_train, y_train)
explainer_forest = dx.Explainer(forest_model, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)

In [171]:
means_basic, df_basic = experiment(explainer)
means_log, df_log = experiment(explainer_log)
means_forest, df_forest = experiment(explainer_forest)

# Reults interpretation

## Top 2 features

As we can see from the plot below, the results are quite different across models. The top-2 features are the same for logistic regression model and xgboost with data scaling. For the random forest model, the top-2 features are `cp` and `thalachh`.



In [293]:
means_basic = means_basic.sort_values(ascending=False)
means_log = means_log.sort_values(ascending=False)
means_forest = means_forest.sort_values(ascending=False)

In [ ]:
means_df = pd.DataFrame({'xgboost': means_basic[:2], 'logistic regression': means_log[:2], 'random forest': means_forest[:2]})
means_plot = px.bar(means_df, title="Feature importance barplot - top features", barmode='group')
means_plot

<img src="fig4.png">

## Bottom 2 features

For the XGBoost model, the bottom-2 features are `chol` and `fbs`. Same for the logistic regression model. The least important features for the random forest model are `chol` and `restecg`.

In [ ]:
means_df = pd.DataFrame({'xgboost': means_basic[11:], 'logistic regression': means_log[11:], 'random forest': means_forest[11:]})
means_plot = px.bar(means_df, title="Feature importance barplot - bottom features", barmode='group')
means_plot

<img src="fig5.png">

# Comparison of different methods for calculating feature importance - XGBoost

Now I will compare the results of the PVI method with the results of the `feature importance` method for the xgboost model. The feature importance in the built-in method for xgboost is calculated for a single decision tree by the amount that each attribute split point improves the performance measure, weighted by the number of observations the node is responsible for. The performance measure is the purity (Gini index) used to select the split points.

In [307]:
built_in = model.feature_importances_

In [198]:
df_compare = pd.DataFrame({'built-in method': built_in, 'PVI': means_no_scaling.drop('_full_model_')})

In [204]:
fig_comparison = px.bar(df_compare, title='Comparison of built-in method and PVI', barmode='group')

## Results interpretation

The built-in method based on Gini impurity is quite different from the PVI method. The Gini impurity shows much more variability across features - the standard deviation is much higher. The PVI method shows much more stable results. However, it can be due to the fact that the PVI method was calculated 100 times for each feature, while the built-in method was calculated only once.
Nevertheless, the top-1 feature is the same - `cp` - chest pain type and the bottom-1 feature is the same - `chol` - cholesterol in mg/dl.

In [ ]:
fig_comparison

<img src="fig6.png">

In [ ]:
data = pd.get_dummies(data, columns=['sex', 'cp', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall'])

y = data['output'] # Target variable
X = data.drop(['output'], axis=1) # Features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xgboost
model = xgboost.XGBClassifier(
    n_estimators=200, 
    max_depth=4, 
)

model.fit(X_train, y_train)

In [ ]:
explainer_shap = dx.Explainer(model, X_test, y_test)

In [319]:
shap_vi = explainer_shap.model_parts(type="shap_wrapper", shap_explainer_type="TreeExplainer", check_additivity=False)

# SHAP variable importance based on tree method

Now we will compare the results of the PVI method with the results of the SHAP variable importance based on tree method. The SHAP tree method is based on the Shapley value.

## Results interpretation

The results are harder to interpret because of one-hot encoding. 

For the SHAP tree method, the most important factor influencing model performance is `caa_0` - the number of major vessels = 0. The variable `oldpeak` which has not appeared in the top-2 features for the PVI method across different models is now the second most important feature. Another thing worth noting is that the feature `chol`, which for many models was the least important feature, is now the third most important feature. What we need to keep in mind, however, is that for variables `oldpeak` and `chol` the values are not consistent with the impact on the model performance. We see high values of `chol` impacting positively on the model performance and negatively (violet dots on both sides of the center line). Same observation for `oldpeak`.

In [ ]:
shap_vi.plot()

<img src="fig7.png">

# Summary

In this notebook, I have shown how to calculate the permutation-based variable importance for the xgboost model. I have also compared the results of the PVI method with the results of the `feature importance` method for the xgboost model and the results of the SHAP variable importance based on tree method.